In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit.topn import RecListAnalysis, ndcg, precision
from lenskit.metrics.predict import rmse
from lenskit import crossfold as xf, batch, topn, util

In [2]:

reviews = pd.read_json("mard/mard_reviews.json" , lines=True)
# want to convert to form: user | item | rating | timestamp
reviews=reviews.drop(['helpful', 'reviewText', 'summary','reviewTime', 'reviewerName'],axis=1)
reviews=reviews[["reviewerID", 'amazon-id', "overall" ,"unixReviewTime"]]


new_column_names = {'reviewerID': 'user', 'amazon-id': 'item', 'overall': 'rating', "unixReviewTime": "timestamp"}
reviews.rename(columns=new_column_names, inplace=True)
len(reviews)

263525

In [3]:


# wrapper class for lenskit evaluation
class evaluator():
    
    
    def __init__(self):
        self.rec_list = RecListAnalysis()
        self.rec_list.add_metric(ndcg, name="ndcg_10", k=10)
        self.rec_list.add_metric(rmse, name="rmse")
        self.rec_list.add_metric(precision, name="precision_10",k=10)
        #self.rec_list.add_metric()
        
        
    
    def add_metric(self, metric_function):
        pass
        
        
    
        
    def evaluate(self, recs, truth):
        '''
        recs: prediction df
        truth: ground truth test data
        '''
        pass
    
    
    
ev = evaluator()
        

In [4]:
def evaluate(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    recs = batch.recommend(fittable, users, 100)
    recs['Algorithm'] = aname
    return recs

In [ ]:
print("Partitioning Data into 5 Folds...")
parts = xf.partition_users(reviews[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2))

for i, (train, test) in enumerate(parts):
    train.to_parquet(f"mard_train-{i}.parquet")
    test.to_parquet(f"mard_test-{i}.parquet")
    


Partitioning Data into 5 Folds...


In [ ]:
algo_ii = knn.ItemItem(20)
all_recs = []
test_data = []

for i in range(5):
    trainset = pd.read_parquet(f"mard_train-{i}.parquet")
    testset = pd.read_parquet(f"mard_test-{i}.parquet")
    
    
    test_data.append(testset)
    all_recs.append(evaluate('ItemItem', algo_ii, trainset, testset))